In [39]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
CALCULATED_CARD_DRAW_DOWNS = dataiku.Dataset("CALCULATED_CARD_DRAW_DOWNS")
CALCULATED_CARD_DRAW_DOWNS_df = CALCULATED_CARD_DRAW_DOWNS.get_dataframe()

CALCULATED_CARD_DRAW_UPS = dataiku.Dataset("CALCULATED_CARD_DRAW_UPS")
CALCULATED_CARD_DRAW_UPS_df = CALCULATED_CARD_DRAW_UPS.get_dataframe()


In [75]:
df_up = CALCULATED_CARD_DRAW_UPS_df
print(len(df_up))
df_up.DRAW_UP_DATE = pd.to_datetime(df_down.DRAW_DOWN_DATE).dt.date
df_up.sort_values(['CUSTOMER'], inplace=True)

df_down = CALCULATED_CARD_DRAW_DOWNS_df
print(len(df_down))
df_down.DRAW_DOWN_DATE = pd.to_datetime(df_down.DRAW_DOWN_DATE).dt.date
df_down.sort_values(['CUSTOMER'], inplace=True)
df_up.head()

322
194


,CUSTOMER,DRAW_UP_DATE,MEAN_DU,STD_DU,ACTIVE_CARD_MAX
0,3GPP,2019-12-01,27.166667,4.239068,35.0
321,3LUM,NaN,12.111111,1.615893,14.0
320,3M (3MUS),NaN,2375.083333,169.146767,2728.0
319,3M COMPANY (3MVR),NaN,360.083333,65.468186,399.0
318,3M COMPANY3MUS,NaN,670.111111,196.492013,963.0


In [102]:
# ===============================================
class Draw_Down_Customer:
    
    def __init__(self, name, draw_down_date, mean_dd, std_dd, active_card_max):

        self._common_words = ['PIZZA', 'MANAGEMENT', 'USA', 'INC', 'US', 'EQUIPMENT', 'MEDICAL', 'SYSTEMS',
                             'ANIMAL', 'HEALTH', 'LLC', 'CORPORATION', 'BRANDS', 'TIRE', 'RUBBER', 'COUNTRY', 'CORP', 
                              'PHARMACY','INC', 'RESTAURANTS', 'CONTAINER', 'AMERICA', 'APPLICATIONS', 'TECHNOLOGY', 
                              'INSURANCE', 'FARM','CREDIT', 'SERVICES', 'SERVICE', 'ACCOUNT', 'GENERAL', 'PARTS', 
                              'INTL', 'FLAVORS', 'HOLDINGS', 'FOOD','INDUSTRIES', 'LP', 'FLEET', 'MEDICAL', 'PHARMA',
                             'GLOBAL', 'PIPELINE', 'WHEELS', 'BIOSCIENCES', 'SSI', 'SPRINGS', 'NORTH', 'MARINE', 'HOLDING', 
                              'TECHNOLOGIES','GROUP', 'PHARMACEUTICAL', 'NA', 'USA', 'COMPANY', 'RAIL', 'PARTNERS', 'BROS', 
                              'CO', 'PHARMACEUTICALS', 'ENERGY', 'DISTRIBUTION', 'DENTAL', 'SPECIALTIES', 'OPERATIONS', 
                              'COMPANY', 'THE', 'MOUNTAIN', 'TRANS', 'FUEL', 'AMERICAN', 'HOMES', 'GAS']
        
        self.CUSTOMER = name
        self.DRAW_DOWN_DATE = draw_down_date
        self.ACTIVE_CARD_MAX = active_card_max
        
        self.MATCHING_CUSTOMERS = []
        self.PERCENT_DIFFERENCE = []
        
        # remove punctuation
        c_str = name.translate(str.maketrans('', '', string.punctuation))
    
        f = c_str.split()
        self.WORD_LIST = []
        for w in f:
            if w not in self._common_words:
                self.WORD_LIST.append(w)
                
    def Match_Draw_Up_Customer(self, name, draw_up_date, mean_du, std_du, active_card_max):
        
        if (self.CUSTOMER == name):
            # exact match, already captured
            return
        
        c_str = name.translate(str.maketrans('', '', string.punctuation))
        
        f = c_str.split()
        
        check_list = []
        for w in f:
            if w not in self._common_words:
                check_list.append(w)
                
        percent_diff = (abs(self.ACTIVE_CARD_MAX - active_card_max) / ((self.ACTIVE_CARD_MAX+active_card_max)/2))*100.
                
        for w_to_check in check_list:
            for w in self.WORD_LIST:
                if w_to_check==w:
                    if not name in(self.MATCHING_CUSTOMERS):
                        self.MATCHING_CUSTOMERS.append(name)
                        self.PERCENT_DIFFERENCE.append(percent_diff)
                        break;
       

idx = 0

_draw_down_customers = []
        
for index, row in df_down.iterrows():
    
    idx+=1
    customer = row['CUSTOMER']
    draw_down_date = row['DRAW_DOWN_DATE']
    mean_dd = row['MEAN_DD']
    std_dd = row['STD_DD']
    active_card_max = row['ACTIVE_CARD_MAX']
    
    c = Draw_Down_Customer(customer, draw_down_date, mean_dd, std_dd, active_card_max)
    _draw_down_customers.append(c)
    
    #if idx>5:
    #    break;
        
for c in _draw_down_customers:
    
    for index_up, row_up in df_up.iterrows():
        
        customer = row_up['CUSTOMER']
        draw_up_date = row_up['DRAW_UP_DATE']
        mean_du = row_up['MEAN_DU']
        std_du = row_up['STD_DU']
        active_card_max = row_up['ACTIVE_CARD_MAX']
        
        c.Match_Draw_Up_Customer(customer, draw_up_date, mean_du, std_du, active_card_max)

match_count = 0
for c in _draw_down_customers:
    
    if len(c.MATCHING_CUSTOMERS)>0:
        match_count+=1
        print(c.CUSTOMER)
        print(c.MATCHING_CUSTOMERS)
        print(c.PERCENT_DIFFERENCE)
        print()
        
print(match_count)

ALKERMES (3ALK)
['ALKERMES INC']
[0.24623115577889448]

AMERICAN HOMES 4 RENT
['AMERICAN HOMES 4 RENT LP']
[0.06587615283267458]

AMGEN USA INC
['AMGEN USA']
[0.18921440567270012]

ARCADIS US
['ARCADIS US INC']
[0.07761194029850746]

ARCHROMA U.S. INC.
['ARCHROMA US INC']
[0.27450980392156865]

ASTELLAS PHARMA
['ASTELLAS PHARMA US INC']
[0.0416]

ATLAS COPCO
['ATLAS COPCO COMPRESSORS LLC3AC2']
[1.9857651245551602]

BALFOUR BEATTY 3BSE
['BALFOUR BEATTY EQUIPMENT', 'BALFOUR BEATTY RENOVATIONS', 'BALFOUR BEATTY3BBR']
[0.49504950495049505, 0.30303030303030304, 1.0731707317073171]

BALFOUR EQUIP
['BALFOUR BEATTY EQUIPMENT', 'BALFOUR BEATTY RENOVATIONS', 'BALFOUR BEATTY3BBR']
[0.5681818181818182, 1.2056737588652482, 0.1087866108786611]

BALFOUR PB
['BALFOUR BEATTY EQUIPMENT', 'BALFOUR BEATTY RENOVATIONS', 'BALFOUR BEATTY3BBR']
[1.6727272727272726, 1.8476190476190477, 1.3949579831932772]

BALFOUR RAIL
['BALFOUR BEATTY EQUIPMENT', 'BALFOUR BEATTY RENOVATIONS', 'BALFOUR BEATTY3BBR']
[1.14576271

In [43]:
customer_down_list = CALCULATED_CARD_DRAW_DOWNS_df.CUSTOMER.unique()
customer_up_list = CALCULATED_CARD_DRAW_UPS_df.CUSTOMER.unique()

In [47]:
CALCULATED_CARD_DRAW_DOWNS_df.head()

,CUSTOMER,DRAW_DOWN_DATE,MEAN_DD,STD_DD,ACTIVE_CARD_MAX
0,3LUM,2019-12-01 00:00:00+00:00,13.000000,0.577350,14.0
193,3US6 (EQUIPMENT),2021-08-01 00:00:00+00:00,36.600000,27.673092,104.0
192,ACORDA THERAPEUTICS,2019-09-01 00:00:00+00:00,129.600000,2.190890,133.0
191,ADF PIZZA MANAGEMENT,2020-12-01 00:00:00+00:00,7.500000,1.623688,12.0
190,ALENT,2020-04-01 00:00:00+00:00,1.333333,0.492366,2.0


In [0]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

POTENTIAL_DUPLICATES_df = ... # Compute a Pandas dataframe to write into POTENTIAL_DUPLICATES


# Write recipe outputs
POTENTIAL_DUPLICATES = dataiku.Dataset("POTENTIAL_DUPLICATES")
POTENTIAL_DUPLICATES.write_with_schema(POTENTIAL_DUPLICATES_df)